In [25]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [26]:
def drop_columns_if_exist(df, columns_to_drop = ["latitude","longitude","elevation","status","STATE", "ID", "entry_id","UNIT"]):
    # Check if each column exists in the DataFrame before dropping it
    columns_to_drop_existing = [col for col in columns_to_drop if col in df.columns]

    # Drop the existing columns from the DataFrame
    df.drop(columns=columns_to_drop_existing, inplace=True)

    return df

In [27]:
def group_and_average_by_minute(df, datetime_column_name):
    df = drop_columns_if_exist(df)
    # Convert the datetime column to datetime format
    df[datetime_column_name] = pd.to_datetime(df[datetime_column_name])

    # Set the datetime column as the DataFrame index for resampling
    df.set_index(datetime_column_name, inplace=True)

    # Resample the DataFrame to group by minute and calculate the average
    df_avg_per_minute = df.resample('1T').mean()

    # Reset the index to convert the datetime column back to a regular column
    df_avg_per_minute.reset_index(inplace=True)

    # Round the datetime column to the nearest minute
    df_avg_per_minute[datetime_column_name] = df_avg_per_minute[datetime_column_name].dt.round('1min')

    return df_avg_per_minute.dropna().reset_index(drop=True)


In [28]:
import pandas as pd

def merge_three_dataframes_by_datetime(df1, df2, df3, datetime_col_df1, datetime_col_df2, datetime_col_df3, timezone=None):
    # Convert the datetime columns to datetime format with the specified timezone (if provided)
    df1["created_at"] = pd.to_datetime(df1['created_at']) + pd.Timedelta(hours=1) # Change time from UTC to WAT
    df1[datetime_col_df1] = pd.to_datetime(df1[datetime_col_df1]).dt.tz_localize(None)
    df2[datetime_col_df2] = pd.to_datetime(df2[datetime_col_df2]).dt.tz_localize(None)
    df3[datetime_col_df3] = pd.to_datetime(df3[datetime_col_df3]).dt.tz_localize(None)

    # Perform the first inner merge between df1 and df2 based on the datetime columns
    merged_df1 = pd.merge(df1, df2, left_on=datetime_col_df1, right_on=datetime_col_df2, how='inner')

    # Drop the redundant datetime column from df2 as it's no longer needed in the first merged DataFrame
    merged_df1.drop(columns=[datetime_col_df2], inplace=True)

    # Perform the second inner merge between the first merged DataFrame and df3 based on their datetime columns
    merged_df = pd.merge(merged_df1, df3, left_on=datetime_col_df1, right_on=datetime_col_df3, how='inner')

    # Drop the redundant datetime column from df3 as it's no longer needed in the final merged DataFrame
    merged_df.drop(columns=[datetime_col_df3], inplace=True)

    return merged_df


In [29]:
alpha_dataset = pd.DataFrame()
data_length_calib_device = []
data_length_low_cost_sensors = []

for i in range(6):
    print("Processing ################# Day {} ############################".format(i+1))
    alpha = pd.read_csv("Day 0{}/Alpha.csv".format(i+1))
    co2 =  pd.read_csv("Day 0{}/CO2.csv".format(i+1))
    co =  pd.read_csv("Day 0{}/CO.csv".format(i+1))
    data_length_calib_device.append(co.shape[0])
    data_length_low_cost_sensors.append(alpha.shape[0])
    
    
    co2 = co2.rename(columns = {"SHOW":"CO2"})
    co = co.rename(columns = {"SHOW":"CO"})
   # co.CO  = co.CO.fillna(0)

    alpha = group_and_average_by_minute(alpha, 'created_at')
    co2 = group_and_average_by_minute(co2, 'TIME')
    co = group_and_average_by_minute(co, 'TIME')

    merged_data = merge_three_dataframes_by_datetime(alpha, co2, co, 'created_at', 'TIME', 'TIME')   
    merged_data = merged_data[30:].reset_index(drop=True)
    alpha_dataset = alpha_dataset.append(merged_data)
    
for i in range(6,9):
    print("Processing ################# Day {} ############################".format(i+1))
    alpha = pd.read_csv("Day 0{}/Alpha.csv".format(i+1))
    co2 =  pd.read_csv("Day 0{}/CO2/CO2.csv".format(i+1))
    co =  pd.read_csv("Day 0{}/CO/CO.csv".format(i+1))
    data_length_calib_device.append(co.shape[0])
    data_length_low_cost_sensors.append(alpha.shape[0])

    co2 = co2.rename(columns = {"SHOW":"CO2"})
    co = co.rename(columns = {"SHOW":"CO"})
    #co.CO  = co.CO.fillna(0)

    alpha = group_and_average_by_minute(alpha, 'created_at')
    co2 = group_and_average_by_minute(co2, 'TIME')
    co = group_and_average_by_minute(co, 'TIME')

    merged_data = merge_three_dataframes_by_datetime(alpha, co2, co, 'created_at', 'TIME', 'TIME')    
    merged_data = merged_data[30:].reset_index(drop=True)
    alpha_dataset = alpha_dataset.append(merged_data)
    

alpha_dataset = alpha_dataset.reset_index(drop=True)
alpha_dataset = alpha_dataset.rename(columns={'SHOW': 'CO2',"field1":"Temperature", "field2":"Humidity", "field3":"MQ7_analog", "field4":"MQ9_analog", "field5":"MG811_analog", "field6":"MQ135_analog", "field7":"MG811_Digital", "field8":"MQ7_Digital"})
alpha_dataset.to_csv("alpha.csv", index=False)
alpha_dataset.tail()

Processing ################# Day 1 ############################
Processing ################# Day 2 ############################
Processing ################# Day 3 ############################
Processing ################# Day 4 ############################
Processing ################# Day 5 ############################
Processing ################# Day 6 ############################
Processing ################# Day 7 ############################
Processing ################# Day 8 ############################
Processing ################# Day 9 ############################


,created_at,Temperature,Humidity,MQ7_analog,MQ9_analog,MG811_analog,MQ135_analog,MG811_Digital,MQ7_Digital,CO2,CO
2809,2023-06-29 14:43:00,30.640,67.470,2395.5,3520.0,1698.5,2071.5,0.0,1.0,583.583333,0.0
2810,2023-06-29 14:44:00,30.665,67.985,2399.5,3503.5,1697.5,2076.5,0.0,1.0,584.500000,0.0
2811,2023-06-29 14:45:00,30.760,70.285,2413.5,3541.5,1692.5,2084.0,0.0,1.0,583.416667,0.0
2812,2023-06-29 14:46:00,30.920,69.880,2409.5,3515.0,1692.0,2083.0,0.0,1.0,585.600000,0.0
2813,2023-06-29 15:40:00,31.620,70.530,2380.0,3460.0,1731.0,2009.0,0.0,1.0,573.166667,1.0


In [30]:
print("sensors", sum(data_length_low_cost_sensors)*4)
print("Calibration device",sum(data_length_calib_device))

print("Estimated Total data points gathered", sum(data_length_low_cost_sensors)*4 + sum(data_length_calib_device))

sensors 30336
Calibration device 12923
Estimated Total data points gathered 43259


In [31]:
Beta_dataset = pd.DataFrame()
for i in range(6):
    print("Processing ################# Day {} ############################".format(i+1))
    Beta = pd.read_csv("Day 0{}/Beta.csv".format(i+1))
    co2 =  pd.read_csv("Day 0{}/CO2.csv".format(i+1))
    co =  pd.read_csv("Day 0{}/CO.csv".format(i+1))
    
    co2 = co2.rename(columns = {"SHOW":"CO2"})
    co = co.rename(columns = {"SHOW":"CO"})
   # co.CO  = co.CO.fillna(0)

    Beta = group_and_average_by_minute(Beta, 'created_at')
    co2 = group_and_average_by_minute(co2, 'TIME')
    co = group_and_average_by_minute(co, 'TIME')

    merged_data = merge_three_dataframes_by_datetime(Beta, co2, co, 'created_at', 'TIME', 'TIME')   
    merged_data = merged_data[30:].reset_index(drop=True)
    Beta_dataset = Beta_dataset.append(merged_data)
    
for i in range(6,9):
    print("Processing ################# Day {} ############################".format(i+1))
    Beta = pd.read_csv("Day 0{}/Beta.csv".format(i+1))
    co2 =  pd.read_csv("Day 0{}/CO2/CO2.csv".format(i+1))
    co =  pd.read_csv("Day 0{}/CO/CO.csv".format(i+1))

    co2 = co2.rename(columns = {"SHOW":"CO2"})
    co = co.rename(columns = {"SHOW":"CO"})
  #  co.CO  = co.CO.fillna(0)

    Beta = group_and_average_by_minute(Beta, 'created_at')
    co2 = group_and_average_by_minute(co2, 'TIME')
    co = group_and_average_by_minute(co, 'TIME')

    merged_data = merge_three_dataframes_by_datetime(Beta, co2, co, 'created_at', 'TIME', 'TIME')    
    merged_data = merged_data[30:].reset_index(drop=True)
    Beta_dataset = Beta_dataset.append(merged_data)
    

Beta_dataset = Beta_dataset.reset_index(drop=True)
Beta_dataset = Beta_dataset.rename(columns={'SHOW': 'CO2',"field1":"Temperature", "field2":"Humidity", "field3":"MQ7_analog", "field4":"MQ9_analog", "field5":"MG811_analog", "field6":"MQ135_analog", "field7":"MG811_Digital", "field8":"MQ7_Digital"})
Beta_dataset.to_csv("Beta.csv", index=False)
Beta_dataset.tail()

Processing ################# Day 1 ############################
Processing ################# Day 2 ############################
Processing ################# Day 3 ############################
Processing ################# Day 4 ############################
Processing ################# Day 5 ############################
Processing ################# Day 6 ############################
Processing ################# Day 7 ############################
Processing ################# Day 8 ############################
Processing ################# Day 9 ############################


,created_at,Temperature,Humidity,MQ7_analog,MQ9_analog,MG811_analog,MQ135_analog,MG811_Digital,MQ7_Digital,CO2,CO
2919,2023-06-29 14:43:00,29.315,68.575,3251.0,4808.0,3515.5,3567.0,0.0,1.0,583.583333,0.0
2920,2023-06-29 14:44:00,29.345,70.175,3232.5,4784.5,3499.0,3555.5,0.0,1.0,584.500000,0.0
2921,2023-06-29 14:45:00,29.390,72.345,3247.5,4787.0,3511.5,3560.5,0.0,1.0,583.416667,0.0
2922,2023-06-29 14:46:00,29.470,71.060,3215.0,4746.0,3497.0,3540.0,0.0,1.0,585.600000,0.0
2923,2023-06-29 15:40:00,30.290,69.660,3143.5,4711.0,3554.5,3587.0,0.0,1.0,573.166667,1.0


In [32]:
Charlie_dataset = pd.DataFrame()
for i in range(6):
    print("Processing ################# Day {} ############################".format(i+1))
    Charlie = pd.read_csv("Day 0{}/Charlie.csv".format(i+1))
    co2 =  pd.read_csv("Day 0{}/CO2.csv".format(i+1))
    co =  pd.read_csv("Day 0{}/CO.csv".format(i+1))
    
    co2 = co2.rename(columns = {"SHOW":"CO2"})
    co = co.rename(columns = {"SHOW":"CO"})
   # co.CO  = co.CO.fillna(0)

    Charlie = group_and_average_by_minute(Charlie, 'created_at')
    co2 = group_and_average_by_minute(co2, 'TIME')
    co = group_and_average_by_minute(co, 'TIME')

    merged_data = merge_three_dataframes_by_datetime(Charlie, co2, co, 'created_at', 'TIME', 'TIME')   
    merged_data = merged_data[30:].reset_index(drop=True)
    Charlie_dataset = Charlie_dataset.append(merged_data)
    
for i in range(6,9):
    print("Processing ################# Day {} ############################".format(i+1))
    Charlie = pd.read_csv("Day 0{}/Charlie.csv".format(i+1))
    co2 =  pd.read_csv("Day 0{}/CO2/CO2.csv".format(i+1))
    co =  pd.read_csv("Day 0{}/CO/CO.csv".format(i+1))

    co2 = co2.rename(columns = {"SHOW":"CO2"})
    co = co.rename(columns = {"SHOW":"CO"})
   # co.CO  = co.CO.fillna(0)

    Charlie = group_and_average_by_minute(Charlie, 'created_at')
    co2 = group_and_average_by_minute(co2, 'TIME')
    co = group_and_average_by_minute(co, 'TIME')

    merged_data = merge_three_dataframes_by_datetime(Charlie, co2, co, 'created_at', 'TIME', 'TIME')    
    merged_data = merged_data[30:].reset_index(drop=True)
    Charlie_dataset = Charlie_dataset.append(merged_data)
    

Charlie_dataset = Charlie_dataset.reset_index(drop=True)
Charlie_dataset = Charlie_dataset.rename(columns={'SHOW': 'CO2',"field1":"Temperature", "field2":"Humidity", "field3":"MQ7_analog", "field4":"MQ9_analog", "field5":"MG811_analog", "field6":"MQ135_analog", "field7":"MG811_Digital", "field8":"MQ7_Digital"})
Charlie_dataset.to_csv("Charlie.csv", index=False)
Charlie_dataset.tail()

Processing ################# Day 1 ############################
Processing ################# Day 2 ############################
Processing ################# Day 3 ############################
Processing ################# Day 4 ############################
Processing ################# Day 5 ############################
Processing ################# Day 6 ############################
Processing ################# Day 7 ############################
Processing ################# Day 8 ############################
Processing ################# Day 9 ############################


,created_at,Temperature,Humidity,MQ7_analog,MQ9_analog,MG811_analog,MQ135_analog,MG811_Digital,MQ7_Digital,CO2,CO
2856,2023-06-29 14:43:00,30.420000,66.320000,3374.000000,3243.000000,2931.000000,2481.0,0.0,1.0,583.583333,0.0
2857,2023-06-29 14:44:00,30.275000,68.545000,3281.500000,3210.500000,2885.500000,2439.5,0.0,1.0,584.500000,0.0
2858,2023-06-29 14:45:00,30.476667,69.843333,3386.333333,3257.333333,2913.333333,2474.0,0.0,1.0,583.416667,0.0
2859,2023-06-29 14:46:00,30.670000,69.190000,3337.000000,3232.000000,2905.000000,2461.0,0.0,1.0,585.600000,0.0
2860,2023-06-29 15:40:00,31.420000,67.975000,3395.500000,3299.500000,2882.000000,2461.5,0.0,1.0,573.166667,1.0


In [33]:
alpha_dataset.head()

,created_at,Temperature,Humidity,MQ7_analog,MQ9_analog,MG811_analog,MQ135_analog,MG811_Digital,MQ7_Digital,CO2,CO
0,2023-05-01 03:29:00,31.415000,63.390,5596.5,1288.0,6870.5,1320.0,0.0,1.0,618.5,0.0
1,2023-05-01 03:30:00,31.445000,63.295,5538.0,1282.5,7035.5,1315.0,0.0,1.0,617.5,0.0
2,2023-05-01 03:31:00,31.500000,63.115,5482.5,1276.0,6906.0,1311.5,0.0,1.0,610.0,0.0
3,2023-05-01 03:32:00,31.510000,63.125,5396.5,1272.5,6879.0,1307.0,0.0,1.0,604.0,0.0
4,2023-05-01 03:33:00,31.523333,62.990,5336.0,1266.0,6895.0,1302.0,0.0,1.0,609.0,0.0


In [34]:
Beta_dataset.head()

,created_at,Temperature,Humidity,MQ7_analog,MQ9_analog,MG811_analog,MQ135_analog,MG811_Digital,MQ7_Digital,CO2,CO
0,2023-05-01 03:30:00,27.790,74.390,3122.0,3854.0,3867.0,3506.0,0.0,1.0,617.5,0.0
1,2023-05-01 03:31:00,27.795,74.335,3110.0,3842.5,3865.5,3499.5,0.0,1.0,610.0,0.0
2,2023-05-01 03:32:00,27.810,74.295,3112.5,3845.5,3868.5,3504.5,0.0,1.0,604.0,0.0
3,2023-05-01 03:33:00,27.795,74.245,3120.0,3853.5,3876.5,3512.0,0.0,1.0,609.0,0.0
4,2023-05-01 03:34:00,27.810,74.245,3109.5,3838.0,3873.5,3516.5,0.0,1.0,617.0,0.0


In [35]:
Charlie_dataset.head()

,created_at,Temperature,Humidity,MQ7_analog,MQ9_analog,MG811_analog,MQ135_analog,MG811_Digital,MQ7_Digital,CO2,CO
0,2023-05-01 03:29:00,28.580,71.865,6016.5,6192.5,4789.5,4420.0,0.0,1.0,618.5,0.0
1,2023-05-01 03:30:00,28.590,71.820,5978.0,6168.0,4794.0,4420.0,0.0,1.0,617.5,0.0
2,2023-05-01 03:31:00,28.580,71.860,5988.5,6154.0,4798.0,4418.5,0.0,1.0,610.0,0.0
3,2023-05-01 03:32:00,28.550,71.995,5990.0,6141.5,4798.0,4414.5,0.0,1.0,604.0,0.0
4,2023-05-01 03:33:00,28.535,71.835,5972.5,6133.5,4798.0,4406.5,0.0,1.0,609.0,0.0


In [36]:
alpha_dataset.isnull().sum()

created_at       0
Temperature      0
Humidity         0
MQ7_analog       0
MQ9_analog       0
MG811_analog     0
MQ135_analog     0
MG811_Digital    0
MQ7_Digital      0
CO2              0
CO               0
dtype: int64